<div>
    <table style="border:2px solid white;" cellspacing="0" cellpadding="0" border-collapse: collapse; border-spacing: 0;>
      <tr> 
        <th style="background-color:white"> <img src="../media/ccal-logo-D3.png" width=225 height=225></th>
        <th style="background-color:white"> <img src="../media/logoMoores.jpg" width=175 height=175></th>
        <th style="background-color:white"> <img src="../media/GP.png" width=200 height=200></th>
        <th style="background-color:white"> <img src="../media/UCSD_School_of_Medicine_logo.png" width=175 height=175></th> 
        <th style="background-color:white"> <img src="../media/Broad.png" width=130 height=130></th> 
      </tr>
    </table>
</div>

<hr style="border: none; border-bottom: 3px solid #88BBEE;">
# **Onco-*GPS* Methodology**
## **Chapter 3. Decomposing Signature and Defining Transcriptional Components**

<div>
    <img src="../media/authors.png" width=900 height=50>
</div>

**Date:** April 17, 2017

**Article:** [*Kim et al.* Decomposing Oncogenic Transcriptional Signatures to Generate Maps of Divergent Cellular States](https://drive.google.com/file/d/0B0MQqMWLrsA4b2RUTTAzNjFmVkk/view?usp=sharing)

**Analysis overview:** In this chapter we will use the KRAS oncogenic signature genes that we generated in chapter 2 and we will decomposed them into non-negative matrix factorization components.

<div>
    <img src="../media/method_chap2.png" width=2144 height=1041>
</div>

To perfrom this analysis we will use as reference dataset 750 samples from the Broad-Novartis Cancer Cell Line Encyclopedia (CCLE) ([*Barretina et al. 2012*](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3320027/)). This dataset, hereafter denoted as $A^{n \times m}$, contains as rows the oncogenic signature genes and  samples representing many instances of the cellular states of interest. 

The decomposition is based on a Non-Negative Matrix Factorization (NMF) algorithm ([*Brunet et al. 2004*](https://www.ncbi.nlm.nih.gov/pubmed/15016911); [*Tamayo et al. 2007*](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1838404/)). These transcriptional components represent summaries of the most coherent gene expression patterns, relevant to the KRAS signature genes across the wide variety of cellular states and contexts represented in the reference dataset. The methodology contains the following analysis tasks:


*	*Normalization*. Normalize the input matrix $A^{n \times m}$ by replacing each gene expression entry by its column rank and obtain matrix $A^{n \times m}_{norm}$.


*	*Matrix Factorization.* Perform a standard non-negative matrix factorization (NMF) ([*Brunet et al. 2004*](https://www.ncbi.nlm.nih.gov/pubmed/15016911); [*Tamayo et al. 2007*](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1838404/), [*Cichocki et al. 2008*](http://www.bsp.brain.riken.jp/publications/2007/Cichocki-Zd-Amar_SPMAG.pdf))

<div>
    <img src="../media/chap3_equation.png" width=200 height=50>
</div>

*    The resulting matrices $W^{n \times k}$ and $H^{k \times m}$ have lower rank than the original matrix $A^{n \times m}_{norm}$ with $k << n, m$.


*	*Model selection.* Find an optimal number of components $k_{c}$ based on the numerical stability of multiple projections using different random seeds  ([*Brunet et al. 2004*](https://www.ncbi.nlm.nih.gov/pubmed/15016911)). The peaks of the cophenetic coefficient represent the more stable decompositions. 


The Matrix $H^{k \times m}$ has the same number of samples as $A^{n \times m}$ but a smaller number of rows and can be interpreted as a summarized version of the original dataset, i.e., one described in the space of the most salient transcriptional programs (components) rather than the original variables (genes). In the KRAS analysis performed below this procedure will produce 9 transcriptional components C1-C9. Restricting the decomposition process to operate using only the signature genes allowed us to emphasize the most relevant oncogene-driven transcriptional space and limit the effects of other transcriptional differences.

Go to the [next chapter (4)](4 Annotating the Transcriptional Components.ipynb).
Back to the [introduction chapter (0)](0 Introduction and Overview.ipynb).

<hr style="border: none; border-bottom: 3px solid #88BBEE;">
### 1. Set up notebook and import Computational Cancer Analysis Library ([CCAL](https://github.com/KwatME/ccal))

In [12]:
from environment2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 2. Read  signature and reference dataset 

Read the oncogenic signature generated in chapter 2

In [13]:
gene_scores = pd.read_csv(
    '../results/kras_signature.txt', sep='\t', index_col=0)

Read the reference dataset (the mRNA expression for the Cancer Cell Line Encyclopedia CCLE Datatset)

In [14]:
reference_dataset = read_gct('../data/ccle_gene_expression.gct')

### 3. Select the subset of rows corresponding to the KRAS signature genes

In [15]:
kras_relevant_genes = get_top_and_bottom_indices(gene_scores, 'score', 500).tolist()

In [16]:
reference_dataset_signature_genes = reference_dataset.loc[
    kras_relevant_genes, :]

### 4. Generate multiple NMF decompositions of the signature genes in the reference dataset

For this reference dataset we will explore NMF decompositions with a range of components from 2 to 12

In [17]:
def drop_na_2d(df, axis='both', how='all'):
    """

    :param df:
    :param axis:
    :param how:
    :return:
    """

    if axis in ('both', 1):
        df = drop_na_1d(df, axis=1, how=how)

    if axis in ('both', 0):
        df = drop_na_1d(df, axis=0, how=how)

    return df

def drop_na_1d(df, axis=0, how='all'):
    """

    :param df:
    :param axis: int;
    :param how:
    :return:
    """

    if axis == 0:
        axis_name = 'column'
    else:
        axis_name = 'row'

    if how == 'any':
        nas = df.isnull().any(axis=axis)
    elif how == 'all':
        nas = df.isnull().all(axis=axis)
    else:
        raise ValueError('Unknown \'how\' \'{}\'; pick from (\'any\', \'all\').'.format(how))

    if any(nas):
        df = df.ix[~nas, :]
        print('Dropped {} {}(s) without any value: {}'.format(nas.sum(), axis_name, nas.index[nas].tolist()))

    return df

In [18]:
drop_na_2d(reference_dataset_signature_genes)

,A101D_SKIN,A172_CENTRAL_NERVOUS_SYSTEM,A204_SOFT_TISSUE,A2058_SKIN,A2780_OVARY,A375_SKIN,A498_KIDNEY,A549_LUNG,A673_BONE,A704_KIDNEY,...,WM88_SKIN,WM983B_SKIN,YAPC_PANCREAS,YD10B_UPPER_AERODIGESTIVE_TRACT,YD38_UPPER_AERODIGESTIVE_TRACT,YD8_UPPER_AERODIGESTIVE_TRACT,YH13_CENTRAL_NERVOUS_SYSTEM,YKG1_CENTRAL_NERVOUS_SYSTEM,ZR751_BREAST,ZR7530_BREAST
Name,,,,,,,,,,,,,,,,,,,,,
SNUPN,5.261501,7.059661,11.158811,7.817427,11.820807,11.240022,7.126872,6.960757,10.390976,6.236368,...,13.192501,8.087785,6.258348,6.661520,4.536234,5.560763,6.210333,8.271501,7.175909,6.428785
CRYL1,6.001259,8.537773,13.566682,12.990962,1.236196,7.149646,7.723379,2.255381,10.562916,34.543797,...,14.771507,6.024782,6.594573,5.609481,5.498563,2.411903,4.218976,6.914605,8.374192,27.119205
TPK1,0.229223,0.463726,0.175605,0.125341,0.672668,0.449700,4.799551,0.215601,0.177125,16.614748,...,0.077223,0.071054,0.550877,0.106682,1.460485,0.665885,0.707363,0.550615,0.384049,0.449813
GEM,9.151510,4.863565,1.767409,1.554120,0.041809,3.857872,10.996700,5.521787,0.715948,3.405040,...,0.606561,19.956139,0.979168,1.931558,1.321913,1.333725,31.679214,29.111918,0.858770,0.416828
AP000318.2,24.239500,11.647090,28.898455,14.783829,12.263256,16.608240,20.108747,5.533933,7.559558,22.458509,...,42.616695,13.216103,12.563704,4.387457,9.033341,4.091899,9.098764,11.372784,4.709253,7.918839
ERCC1,47.915573,16.245222,73.207687,36.787075,29.050751,76.592873,21.926508,37.072163,39.306999,20.651030,...,33.543468,45.062149,11.316092,15.469670,46.310505,13.756838,39.154526,42.092415,20.400351,25.180040
RSAD1,13.115775,9.297009,10.283496,28.110376,11.451027,21.056053,9.842514,15.915138,13.617535,9.818786,...,14.502257,19.638004,11.949345,8.369070,7.963938,4.057518,14.362863,18.944107,17.264780,25.035215
AEBP1,0.625284,0.082404,0.089869,57.757500,0.049206,8.805905,0.081424,0.014547,13.745560,0.086328,...,2.461028,16.495352,0.518982,0.284800,0.245312,0.163761,16.896368,3.951565,0.339828,0.046778
LEPRE1,44.828781,39.391525,49.814144,33.407177,20.461527,21.973753,30.052139,10.396082,16.468138,4.862354,...,21.103979,32.880348,4.531857,13.651402,8.861572,85.818474,61.926247,41.339233,2.509074,2.248291


In [19]:
nmf_results, cophenetic_correlation_coefficients = define_components(
    reference_dataset_signature_genes,
    ks=range(2, 11),
    directory_path='../results',
    n_clusterings=30,
    random_seed=6137)

NameError: name 'drop_nan_slices' is not defined

### 5. Choose optimal decomposition (k=9) and define NMF components

Looking at the results of the prior computation we can see that there is an optimal solution for k= 9 components. The code below reads the $W$ and $H$ matrices corresponding to 9 components.

Here  we will also relabeled the component to match the labeling used in the article. The reason for the different labeling is because the original component numbers used in the article were generated using an earlier version of the Onco-*GPS* code written in the R language that used different random numbers to initiallize the $W$ and $H$ matrices.

In [ ]:
w_matrix = ccal.read_gct('../results/nmf_cc/nmf/nmf_k9_w.gct')

In [ ]:
h_matrix = ccal.read_gct('../results/nmf_cc/nmf/nmf_k9_h.gct')

In [ ]:
indices = ['C1', 'C3', 'C9', 'C8', 'C6', 'C7', 'C5', 'C2',
           'C4']  # Relabel components to have same names as in the paper

In [ ]:
h_matrix.index = indices

In [ ]:
w_matrix.columns = indices

In [ ]:
ccal.write_gct(h_matrix, '../results/nmf_cc/nmf/nmf_k9_h.gct')

In [ ]:
ccal.write_gct(w_matrix, '../results/nmf_cc/nmf/nmf_k9_w.gct')

Plot the W and H matrices for k=9 components (this H matrix is shown on the right of Fig 3 in the article)

In [ ]:
ccal.oncogps.plot_nmf(w_matrix=w_matrix, h_matrix=h_matrix)

### 6. Compare the NMF components with each other using the information coefficient
This computation generates an association matrix that compares the component profiles against each other using the Information Coefficient. This is figure S6A in the article and shows that the components are distinct.

In [ ]:
CM = ccal.association.make_comparison_panel(
    matrix1=h_matrix,
    matrix2=h_matrix,
    axis=1,
    matrix1_label='Component',
    matrix2_label='Component')